In [0]:
from __future__ import print_function  # needed for Python 2
from __future__ import division        # needed for Python 2
import csv                               # csv reader
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from random import shuffle
from sklearn.pipeline import Pipeline

In [2]:
# !pip install nltk
# !pip install sklearn
import nltk
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [0]:
# load data from a file and append it to the rawData

def loadData(path, Text=None):
    with open(path,'r') as f:
        reader = csv.reader(f, delimiter='\t')
        reader.next() # ignore header
        for line in reader:
            (Id, Text, Rating, VerPurchase, Label) = parseReview(line)
            rawData.append((Id, Text, Rating, VerPurchase, Label))
            preprocessedData.append((Id, preProcess(Text), Rating, VerPurchase, Label))
            #print(preProcess(Text))

        
def splitData(percentage):
    dataSamples = len(rawData)
    halfOfData = int(len(rawData)/2)
    trainingSamples = int((percentage*dataSamples)/2)
    for (index, Text,_,_, Label) in rawData[:trainingSamples] + rawData[halfOfData:halfOfData+trainingSamples]:
        trainData.append((toFeatureVector(preProcess(Text),index),Label))
    for (index, Text,_,_, Label) in rawData[trainingSamples:halfOfData] + rawData[halfOfData+trainingSamples:]:
        testData.append((toFeatureVector(preProcess(Text),index),Label))

In [0]:
# QUESTION 1
import csv,re
# Convert line from input file into an id/text/label tuple
def parseReview(reviewLine):
    """
    reviewLine is a list
    """
    # Should return a triple of an integer, a string containing the review, and a string indicating the label
    doc_id = int(reviewLine[0])
    label = reviewLine[1]
    review_text = reviewLine[8]
    #print((doc_id, review_text, label))
    return (doc_id, review_text, reviewLine[2], reviewLine[3] ,label)


In [0]:
# TEXT PREPROCESSING AND FEATURE VECTORIZATION
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import words
stopwords = stopwords.words('english')
porter = PorterStemmer()

# Input: a string of one review
def preProcess(text):
    # Should return a list of tokens    
    # CHANGE THE CODE BELOW
    # # word tokenisation
    # text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", text)
    # text = re.sub(r"([.,;:!?'\"“\(])(\w)", r"\1 \2", text)
    # #print "tokenising:", text
    # tokens = re.split(r"\s+",text)
    # # normalisation
    # text = re.sub(r"(\S)\1\1+",r"\1\1\1", text)
    text = text.lower()
    text = re.sub('[^A-Za-z0-9]+',' ',text)
    tokens = text.split(' ')
    # stop word removal
    tokens = [w for w in tokens if w not in stopwords]
    # Stemming
    tokens = [porter.stem(w) for w in tokens]
    # tokens = [porter.stem(w) if porter.stem(w) in set(words.words()) else w for w in tokens ] 
    # remove speacial char
    tokens = [w for w in tokens if w is not '' ]

    return tokens



In [6]:
# QUESTION 2
rawData = []
preprocessedData = []
loadData('amazon_reviews.txt')
print(preprocessedData[0:10])
featureDict = {} # A global dictionary of features
import collections
alltokens =[]
for i in preprocessedData:
    alltokens.extend(i[1])
featureDict = collections.Counter(alltokens)

[(1, ['least', 'think', 'product', 'save', 'day', 'keep', 'around', 'case', 'need', u'someth'], '4', 'N', '__label1__'), (2, ['lithium', u'batteri', u'someth', 'new', u'introduc', 'market', u'averag', u'develop', 'cost', u'rel', 'high', 'stallion', u'compromis', u'qualiti', u'provid', 'us', 'best', 'low', 'cost', 'br', u'mani', 'built', u'technic', u'assist', 'act', 'like', 'sensor', 'particular', 'fort', u'batteri', u'keep', 'phone', u'charg', u'work', u'everi', u'voltag', 'high', u'voltag', 'never', u'risk'], '4', 'Y', '__label1__'), (3, [u'purchas', 'swing', u'babi', '6', u'month', u'pretti', 'much', 'grown', 'loud', 'swing', 'well', u'beauti', 'though', 'love', u'color', 'lot', u'set', 'think', 'worth', 'money'], '3', 'N', '__label1__'), (4, [u'look', u'inexpens', 'desk', 'calcolatur', u'work', u'everyth', 'need', u'issu', u'tilt', u'slightli', 'one', 'side', 'hit', u'key', u'rock', u'littl', 'bit', 'big', 'deal'], '4', 'N', '__label1__'), (5, ['use', 'twice', 'week', u'result', 'g

In [7]:
import numpy as np
from scipy import stats
length_of_token = []
for i in preprocessedData:
    length_of_token.append(len(i[1]))

print("Avg length of tokens", np.mean(length_of_token))
print("Median of tokens",np.median(length_of_token))
print("Mode of tokens",stats.mode(length_of_token))



Avg length of tokens 35.214285714285715
Median of tokens 22.0
Mode of tokens ModeResult(mode=array([12]), count=array([1013]))


In [0]:
import math
def toFeatureVector(tokens,index=None):
    # Should return a dictionary containing features as keys, and weights as values
    adict = {}
    total_no_of_words = sum(featureDict.values())
    total_no_of_reviews = len(rawData)

    for i in tokens[:22]:
      count = 0
      for line in rawData:
        if i in line:
          count = count+1
      adict[i] = (featureDict[i]/total_no_of_words)*math.log(float(1 + total_no_of_reviews) / (1 + count))
    if index is not None:
      for i in preprocessedData:
        if i[0] == index:
          adict['raiting'] = i[2]
          adict['verPur'] = 1 if i[3] == 'Y' else 0
    return adict

In [26]:
token = ['least', 'think', 'product', 'save', 'day', 'keep', 'around', 'case', 'need', u'someth']
toFeatureVector(token,index=1)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  # This is added back by InteractiveShellApp.init_path()


{'around': 0.01627094162211614,
 'case': 0.017684050695997197,
 'day': 0.025853166961195294,
 'keep': 0.020294937937263148,
 'least': 0.005652436295524218,
 'need': 0.038598064989436805,
 'product': 0.06018498836567692,
 'raiting': '4',
 'save': 0.006459927194884821,
 u'someth': 0.01531541072453943,
 'think': 0.020792890658535516,
 'verPur': 0}

In [0]:
# TRAINING AND VALIDATING OUR CLASSIFIER
def trainClassifier(trainData):
    print("Training Classifier...")
    pipeline =  Pipeline([('svc', LinearSVC())])
    return SklearnClassifier(pipeline).train(trainData)

In [0]:
# QUESTION 3
# from sklearn.model_selection import cross_val_score
# scores = cross_val_score(lr, boston.data, boston.target, cv=7, scoring='neg_mean_squared_error')
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
def crossValidate(dataset, folds):
    shuffle(dataset)
    cv_results = []
    foldSize = int(len(dataset)/folds)
    for i in range(0,len(dataset),foldSize):
      valD = dataset[i:i+foldSize]
      testD = dataset[:i]+dataset[i+foldSize:] #list(set(dataset)-set(dataset[i:i+foldSize]))
      classi = trainClassifier(testD)
      print('predicting')
      y_true = map(lambda t: t[1], valD)
      y_pred = predictLabels(valD,classi)
      print(precision_recall_fscore_support(y_true, y_pred, average='macro'))
      print(accuracy_score(y_true,y_pred))
      # trainClassifier()
      # predictLabels()
      # break; # Replace by code that trains and tests on the 10 folds of data in the dataset
    return cv_results

In [30]:
# z=['a','b','c','d','e','f','g','h']
# for i in range(0,len(z),2):
#   print(z[i:i+2],list(set(z)-set(z[i:i+2])))
#   print(z[i:i+2],z[:i]+z[i+2:])
#   print('\n')

tokens = ['a','b','c','d','e','f','g','h']
tokens[:4]

['a', 'b', 'c', 'd']

In [0]:
# PREDICTING LABELS GIVEN A CLASSIFIER

def predictLabels(reviewSamples, classifier):
    # return classifier.classify_many(map(lambda t: toFeatureVector(preProcess(t[1])), reviewSamples))
    return classifier.classify_many(map(lambda t: t[0], reviewSamples))



def predictLabel(reviewSample, classifier):
    return classifier.classify(toFeatureVector(preProcess(reviewSample)))

In [0]:
# MAIN

# loading reviews
rawData = []          # the filtered data from the dataset file (should be 21000 samples)
preprocessedData = [] # the preprocessed reviews (just to see how your preprocessing is doing)
trainData = []        # the training data as a percentage of the total dataset (currently 80%, or 16800 samples)
testData = []         # the test data as a percentage of the total dataset (currently 20%, or 4200 samples)

# the output classes
fakeLabel = 'fake'
realLabel = 'real'

# references to the data files
reviewPath = 'amazon_reviews.txt'

## Do the actual stuff
# We parse the dataset and put it in a raw data list
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing the dataset...",sep='\n')
loadData(reviewPath) 
# We split the raw dataset into a set of training data and a set of test data (80/20)
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing training and test data...",sep='\n')
splitData(0.8)
# We print the number of training samples and the number of features
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Training Samples: ", len(trainData), "Features: ", len(featureDict), sep='\n')

Now 0 rawData, 0 trainData, 0 testData
Preparing the dataset...
Now 21000 rawData, 0 trainData, 0 testData
Preparing training and test data...


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  # This is added back by InteractiveShellApp.init_path()


In [14]:
crossValidate(trainData, 10)


Training Classifier...
predicting
(0.6008302687694803, 0.5856555634301913, 0.5680510581279862, None)
0.5833333333333334
Training Classifier...
predicting
(0.5668964744128693, 0.5659718432827676, 0.5641691921920043, None)
0.5654761904761905
Training Classifier...
predicting
(0.5889738695568992, 0.585050505050505, 0.5814378274055694, None)
0.5869047619047619
Training Classifier...
predicting
(0.5780924935511105, 0.5567836403230197, 0.5214473697649795, None)
0.5517857142857143
Training Classifier...
predicting
(0.5882509932321712, 0.5804169030062394, 0.5728128356952578, None)
0.5839285714285715
Training Classifier...
predicting
(0.5698290298710909, 0.5686100828235987, 0.5666280854176755, None)
0.5684523809523809
Training Classifier...
predicting
(0.5783429533429534, 0.5775649102616254, 0.5767343172054913, None)
0.5779761904761904
Training Classifier...
predicting
(0.5589057680666456, 0.5463389192498695, 0.520169394200402, None)
0.5452380952380952
Training Classifier...
predicting
(0.57846

[]